In [113]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [114]:
url = "https://business-mail.jp/example"
page = requests.get(url)
soup = BeautifulSoup(page.text, "html")

links = soup.find_all("a")
wants = [
    "https://business-mail.jp/cat_example/outside/",
    "https://business-mail.jp/cat_example/office/",
    "https://business-mail.jp/tag_example/"
]
filtered_links = [a['href'] for a in soup.find_all('a') if any(a['href'].startswith(y) for y in wants)]

data = []

for link in filtered_links:
    # get current topic page
    topic_page = requests.get(link)
    topic_soup = BeautifulSoup(topic_page.text, "html")

    # get all example links from current topic
    topic_example_links = [a["href"] for a in topic_soup.find_all("a") \
        if a["href"].startswith("https://business-mail.jp/example/")]
    topic_example_links = [x for x in topic_example_links if x != "https://business-mail.jp/example/"]

    # get title and text from each example
    for example_link in topic_example_links:
        example_page = requests.get(example_link)
        example_soup = BeautifulSoup(example_page.text, "html.parser")
        example_title = example_soup.find_all("h1")[0].get_text()
        example_email = [p.get_text(separator="\n") for p in example_soup.find_all('p')][0]
        data.append([example_link, example_title, example_email])

In [185]:
# clean up and save as csv
columns = ["link", "title", "email"]
df = pd.DataFrame(data, columns=columns)
df = df.drop_duplicates("email").reset_index().drop("index", axis=1)

def truncate_signature(text):
    index = text.find("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    if index != -1:
        return text[:index]
    return text

df["email_no_sig"] = df["email"].apply(lambda x: truncate_signature(x))

# df.to_csv("japanese_business_email_examples.csv")


In [190]:
" ".join(df["email_no_sig"].tolist())

'株式会社●●●●\n\r\n●●●●様\n\n\n\r\n大変ご無沙汰しております。\n\r\n一般社団法人日本ビジネスメール協会の山田太郎です。\n\n\n\r\nこの度、弊社では過去に資料請求をしていただいた方に対し、\n\r\nメールマガジンでフォローをさせていただくことになりました。\n\n\n\r\n最新の研修動向、メールの上手な書き方\n\r\nビジネスメール実態調査の最新データなど、\n\r\nプロとしての切り口をお楽しみいただけたらと思います。\n\n\n\r\n配信は、毎週月曜日の11時頃を予定しております。\n\n\n\n\n\r\nなお、いきなり送信してしまうと失礼になりますので、\n\r\n大変お手数ですが、不要な方は以下のURLから解除をお願い致します。\n\n\n\r\n★URLをここに貼り付ける★\n\n\n\n\n\r\nそれでは、来週の月曜日をお待ちください。\n\n\n\r\n引き続きよろしくお願いいたします。\n\n\n\n\n\r\n 株式会社●●●●\n\r\n●●●●様\n\n\n\r\nお世話になっております。\n\r\n一般社団法人日本ビジネスメール協会の山田太郎です。\n\n\n\r\n本日、お電話で以下の講座について、ご予約のキャンセルを承りました。\n\n\n\r\n【講座名】●●\n\r\n【日時】2019年03月16日（土）10:00〜12:00\n\r\n【受講人数／受講料】1名\u3000●●円（税込）\n\n\n\r\nお振り込みはまだのため、返金等はございません。\n\n\n\r\nまた機会がございましたら、お申し込みいただけますと幸いです。\n\n\n\r\nご不明な点やご質問がございましたら、お気軽にご連絡ください。\n\r\nそれでは、今後ともよろしくお願いいたします。\n\n\n\r\n 株式会社●●●●\n\r\n●●●●様\n\n\n\r\nお世話になっております。\n\r\n一般社団法人日本ビジネスメール協会の山田太郎です。\n\n\n\r\n住居表示変更のお知らせです。\n\n\n\r\n当協会の住居表示が、●●●●年●月●日（●）より、\n\r\n以下に変更いたします。\n\n\n\r\n-------------------------------\n\r\n（新）\n\r\n〒